3. Utilice las clases creadas para construir un modelo de predicción para el problema de diabetes de Sklearn. Cumpla además los siguientes requisitos:

a) Solo puede trabajar con los 5 atributos más relevantes según PearsonmRMR.

b) Deberá de construirse un pipeline que incluya la normalización de los datos, selección de atributos y el modelo de KNN regresión.

c) Utilice validación cruzada con 3 folds para elegir el mejor valor de k entre los valores {1, 3, 5, 7, 15}.

d) Deberá reportar el error cuadrático medio (MSE) estimado

In [1]:
from sklearn.datasets import load_diabetes
data = load_diabetes()

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# (1) Separar atributos de las etiquetas
X, y = data.data, data.target

# (2) Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42
)

In [3]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
import pandas as pd
from PearsonmRMR import PearsonmRMR
from KNNRegression import KNNRegressor

resultados = []
k = 5

pipe = Pipeline([
    ('select', PearsonmRMR(n_features=k)),
    ('scaler', MinMaxScaler()),
    ('knn', KNNRegressor())
])

param_grid = {'select__n_features': [5],
              'knn__k': [3, 5, 7, 9, 11, 13, 15]}
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

gridK = GridSearchCV(pipe, param_grid, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)
gridK.fit(X_train, y_train)

best_K = gridK.best_params_['select__n_features']
print(f"Mejor K: {best_K}")
best_k = gridK.best_params_['knn__k']
mse_cv = -gridK.best_score_  # positivo CV
y_pred_test = gridK.best_estimator_.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"MSE CV: {mse_cv:.2f}, MSE Test: {mse_test:.2f}")

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/model_selection/_split.py:784: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_of_target_y = type_of_target(y)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Mejor K: 5
MSE CV: 3330.30, MSE Test: 3026.28
